## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,41.70,95,0,4.25,clear sky
1,1,Bethel,US,41.3712,-73.4140,72.75,71,0,0.00,clear sky
2,2,Rio Gallegos,AR,-51.6226,-69.2181,41.63,88,100,10.65,overcast clouds
3,3,Carnarvon,AU,-24.8667,113.6333,68.00,51,59,9.62,broken clouds
4,4,Najran,SA,17.4924,44.1277,79.47,40,37,5.39,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter the minimum temperature you would like for your trip "))
max_temp = float(input("Please enter the maximum temperature you would like for your trip "))

Please enter the minimum temperature you would like for your trip 75
Please enter the maximum temperature you would like for your trip 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Najran,SA,17.4924,44.1277,79.47,40,37,5.39,scattered clouds
9,9,The Valley,AI,18.2170,-63.0578,80.82,78,1,19.77,clear sky
15,15,Avarua,CK,-21.2078,-159.7750,75.25,94,75,11.50,drizzle
18,18,Vaini,TO,-21.2000,-175.2000,80.76,78,20,5.75,few clouds
23,23,Kapaa,US,22.0752,-159.3190,84.18,78,100,17.27,overcast clouds
32,32,Hilo,US,19.7297,-155.0900,81.28,88,100,16.11,moderate rain
34,34,Hovd,MN,48.0056,91.6419,80.85,22,93,3.04,overcast clouds
35,35,Cayenne,GF,4.9333,-52.3333,76.08,1,100,0.00,overcast clouds
47,47,Naze,JP,28.3667,129.4833,79.02,80,100,20.80,overcast clouds
52,52,Bonthe,SL,7.5264,-12.5050,76.96,86,100,9.46,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Najran,SA,79.47,scattered clouds,17.4924,44.1277,
9,The Valley,AI,80.82,clear sky,18.2170,-63.0578,
15,Avarua,CK,75.25,drizzle,-21.2078,-159.7750,
18,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,
23,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,
32,Hilo,US,81.28,moderate rain,19.7297,-155.0900,
34,Hovd,MN,80.85,overcast clouds,48.0056,91.6419,
35,Cayenne,GF,76.08,overcast clouds,4.9333,-52.3333,
47,Naze,JP,79.02,overcast clouds,28.3667,129.4833,
52,Bonthe,SL,76.96,moderate rain,7.5264,-12.5050,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Najran,SA,79.47,scattered clouds,17.4924,44.1277,محمد هادي ال فطيح
9,The Valley,AI,80.82,clear sky,18.2170,-63.0578,CeBlue Villas & Beach Resort
15,Avarua,CK,75.25,drizzle,-21.2078,-159.7750,Paradise Inn
18,Vaini,TO,80.76,few clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Kapaa,US,84.18,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))